# Вводная

В рамках прохождения курса по математики, темы Линейной алгебры, был очень впечатлен таким простым / не простым методом как матричная факторизация. В курсе давался массив данных 20 на 30, и на нем показывался метод. Прикольно. Но захотелось чего-то большего. Захотелось своими руками воспроизвести все этапы, от сбора данных, до получения резульатов самой модели.

Интересный факт.  
В 2006 году Netflix, в ходе улучшений своих рекомендательных систем, нашла новый подход, основанный на матричной факторизации. На самом деле методу было уже пара веков, но Netflix смогла его применить к бизнес задаче, заработать много тысяч денег, и стать теми кем мы все их знаем. В истории даже остался термин Netflix Matrix Factorization *(Матричная факторизация Нетфликса)*.

В общем захотелось посмотреть что это такое. Данные парсил с КиноПоиска. Собрал данные по 300 пользователям, 16000 фильмам. Получилось ~130.000 оценок.  

Честно скажу, показатели можно выкрутить еще сильнее, модель можно улучшать и оптимизировать. Направлений много. В проекте оставил те, что дали лучший результат.

# Описание

В данном проекте будем исследовать подход к построению рекомендательной системы с использованием метода матричной факторизации.  

Наши данные включают оценки пользователей для различных фильмов, что позволит нам предсказать будущие предпочтения пользователей.

# Подготовка данных

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('/kaggle/input/data-kp/data_kp_total.csv',index_col=0)

df.head()

In [ ]:
df.shape

In [ ]:
df.rename(columns={'number_voters': 'number_ratings'}, inplace=True)

In [ ]:
df.columns

In [ ]:
df = df[['user_name', 'movie_ru', 'movie_year', 'time_movie',  'rating', 'number_ratings', 'vote', 'date']]
df = df.reset_index(drop=True)
df.tail()

- `rating` -  рейтинг фильма на КиноПоиске
- `number_ratings` -  количество проголосовавших за этот рейтинг
- `vote` -  оценка пользователя

# Предобработка

In [ ]:
# количество дублей
df.duplicated().sum()

In [ ]:
df.duplicated(['user_name','movie_ru']).sum()

In [ ]:
# количество пропусков
df.isna().sum()

In [ ]:
# поправляем индексы
df = df.reset_index(drop=True)
df

In [ ]:
# распределение по оценкам
df['vote'].value_counts()

In [ ]:
plt.hist(df['vote'],bins=10)
plt.show()

- единиц больше чем двоек, это не естественно для нормального распределения

In [ ]:
# оценки равные 1
df.query('vote == 1')

- скорее всего единицы ставят чтобы выразить негодование, и чтобы намеренно опустить общую оценку фильма вниз  
- такие оценки не оценивают фильм, а скорее выражают позицию
  

- чтобы не путать модель, уберем единицы из набора

In [ ]:
# удалим самые низкие оценки
df = df.query('vote != 1')
df.shape[0]

In [ ]:
# посчитаем сколько фильмов получили по сколько оценок
bar = df['movie_ru'].value_counts()
bar[:15]

In [ ]:
# посчитаем количества по количеству оценок у фильмов
# сколько фильмов имеет по сколько оценок
bar = df['movie_ru'].value_counts().reset_index().rename(columns={'count': 'number_votes'})
bar['number_votes'].value_counts().reset_index().rename(columns={'count': 'number_movies'})

- `number_votes` - количество оценок
- `number_movies` - количество фильмов
- 6468 фильмов имеют всего одну оценку
- фильмы оцененные только двумя пользователями - 2238
- самый оцененный фильм в нашем наборе имеет 138 оценок - **"Дюна"** 2021 года


---

Для будущей передсказательной модели необходимо чтобы объекты (фильмы) имели какое-то минимальное количество наблюдений (оценок)  

Уберем из набора фильмы с низким количеством оценок

In [ ]:
# установим минимальное количество оценок
min_vote = 5

In [ ]:
# фильмы с малым количеством оценок
(df['movie_ru'].value_counts().reset_index()).query('count <= @min_vote')

In [ ]:
# создаем список фильмов с малым количеством оценок
movie_del = list((df['movie_ru'].value_counts().reset_index()).query('count <= @min_vote')['movie_ru'])
movie_del[:10]

In [ ]:
# размер списка
len(set(movie_del))

In [ ]:
# вырезаем из массива все фильмы с малым количеством оценок
df = df.query('movie_ru not in @movie_del')
df.shape

---

In [ ]:
# количество уникальных фильмов
df['movie_ru'].nunique()

In [ ]:
# количество юзеров
df['user_name'].nunique()

In [ ]:
# общее количество оценок
df.shape[0]

In [ ]:
# уровень разряженности полной матрицы
df.shape[0] / (df['movie_ru'].nunique() * df['user_name'].nunique())

# Делим данные на test / train

In [ ]:
train, test = train_test_split(df, test_size=0.1, random_state=42)
print(f'train: {len(train)}')
print(f'test:  {len(test)}')

In [ ]:
train.shape

In [ ]:
test.shape

# Собираем модель

In [ ]:
# пересоберем наш тренеровочный набор в сводную таблицу
# по строкам будут пользователи, по колонкам фильмы, в значениях оценки
pivot = train.pivot_table(columns='movie_ru', index='user_name', values='vote')
pivot

In [ ]:
# заменим пустые значения нулями
pivot = pivot.fillna(0)

In [ ]:
# преобразуем сводную таблицу в матрицу рейтингов - R
R = np.array(pivot)
R

In [ ]:
R.shape

 - **R - матрица рейтингов**  
Это матрица, где строки соответствуют пользователям, а столбцы – фильмам. Каждое значение в матрице представляет собой рейтинг, который пользователь поставил конкретному фильму. Если пользователь не оценил фильм, то соответствующая ячейка остается пустой.
 - **R_binary - Бинарная матрица**  
Это матрица с такой же структурой, как и матрица рейтингов `R`, но вместо реальных значений рейтингов содержит бинарные значения. True указывает на то что пользователь оценил фильм, а False – что не оценил

In [ ]:
# создаем бинарную матрицу для отслеживания ненулевых рейтингов
R_binary = R > 0
R_binary



 - **K - количество скрытых факторов**  
Это параметр, который определяет количество скрытых факторов, используемых для матричной факторизации. Скрытые факторы представляют собой абстрактные характеристики, которые могут объяснить предпочтения пользователей и свойства фильмов.  

- **P - матрица пользователь-фактор**  
Это матрица размером 𝑚 × K, где 𝑚 – количество пользователей, а K – количество скрытых факторов. Каждая строка в матрице P представляет собой вектор скрытых факторов для одного пользователя. Эти векторы описывают предпочтения пользователей в терминах скрытых факторов.  
  
- **Q - матрица фактор-элемент**  
Это матрица размером 𝑛 × K, где 𝑛 – количество фильмов, а K – количество скрытых факторов. Каждая строка в матрице Q представляет собой вектор скрытых факторов для одного фильма. Эти векторы описывают свойства фильмов в терминах скрытых факторов.

In [ ]:
# количество скрытых факторов
K = 80

# инициализизуем матрицы пользователь-фактор и фактор-элемент случайными значениями
P = np.random.rand(R.shape[0], K)
Q = np.random.rand(K, R.shape[1])

print('P:', P.shape)
print('Q:', Q.shape)

---

**Матричная факторизация**  
Основная идея заключается в разложении исходной матрицы рейтингов R на две матрицы меньшей размерности P и Q так, чтобы их произведение приближало исходную матрицу R.  
  
Для нахождения оптимальных значений матриц P и Q будем использовать метод градиентного спуска

In [ ]:
steps = 400      # количество шагов для обучения
lr = 0.001       # скорость обучения
lambda_ = 0.015  # параметр регуляризации

tolerance = 1e-6        # порог терпимости для критерия остановки
previous_loss = np.inf  # предустановленная ошибка - бесконечность

for step in range(steps):
    for k in range(K):
        # обновляем P и Q с использованием векторизации, для P[:, k] и Q[k, :]
        P[:, k] += lr * ((R - np.dot(P, Q)) * R_binary).dot(Q[k, :].T) - lambda_ * P[:, k]
        Q[k, :] += lr * ((R - np.dot(P, Q)) * R_binary).T.dot(P[:, k]) - lambda_ * Q[k, :]

    # расчет среднеквадратической ошибки loss (метод MSE)
    squared_error = (R_binary * (R - np.dot(P, Q)))**2
    loss = np.sum(squared_error) / np.count_nonzero(R_binary)

    # вывод прогресса каждые 10 шагов
    if (step + 1) % 10 == 0:
        print(f"Step {step + 1}, loss: {loss.round(5)}")

    # остановка, если разница в ошибке меньше заданного порога
    if np.abs(previous_loss - loss) < tolerance:
        break

    previous_loss = loss

In [ ]:
print("Предсказанная матрица:")
print((P @ Q).round(1)[:10,:10])
print("\nОригенальная матрица:")
print(R[:10,:10])

In [ ]:
# создаем переменную approx - матрицу предсказаний
approx = P @ Q

In [ ]:
# заменяем в предсказанной матрице известные рейтинги из исходной матрицы
approx[R_binary] = R[R_binary]
approx.round(1)[:10,:10]

In [ ]:
# пересобераем approx в DataFrame model, с названиями фильмов и user_name
model = pd.DataFrame(columns=pivot.columns, index=pivot.index, data=approx)
model

- `model` - это наша предсказательная модель
- это таблица с пользователями в строках, и фильмами в столбцах
- в значениях стоят предсказанные оценки по всем фильмам
- кроме тех случаев, когда пользователь ставил оценку
- оценку можно заметить, числа с 6 нулями после точки

# Оценка модели

In [ ]:
# преобразуем model в вертикальную таблицу
model_long = model.reset_index().melt(id_vars=['user_name'], var_name='movie_ru', value_name='vote_predict')
model_long.rename(columns={'index': 'user_name'}, inplace=True)
model_long

- теперь `model` представлена в виде в вертикальной таблицы `model_long`
- каждому пользователю, на каждый фильм предсказана оценка `vote_predict`
- всего 1.388.955 оценок

In [ ]:
# выведем тренировочные данные
train_users = train['user_name'].nunique()
train_movies = train['movie_ru'].nunique()
train_votes = train.shape[0]
sparse_matrix = train_votes / model_long.shape[0]

print('ТРЕНИРОВОЧНЫЕ ДАННЫЕ: \n')
print(f'train_users:    {train_users}')
print(f'train_movies:   {train_movies}')
print(f'train_votes:    {train_votes}')
print(f'sparse_matrix:  {round(sparse_matrix,3)}')

In [ ]:
# чтобы оценить результат, объединаяем model_long с test
res = test.merge(model_long, on=['user_name', 'movie_ru'], how='inner')
result = res[['user_name','movie_ru','rating','vote_predict','vote']].copy()
result

- мы объеденили тестовую выборку `test` с нашей `model_long` по inner пересечению, по `'user_name'`, `'movie_ru'` колонкам
- получаем таблицу `result` :
 - колонка `rating` - рейтинг на КиноПоиске
 - колонка `vote_predict` - предсказанная оенка модели
 - колонка `vote` - оценки пользователей, на которых модель не обучалась

In [ ]:
# считаем тестовые характеристики
users = result['user_name'].nunique()
movies = result['movie_ru'].nunique()
votes = result.shape[0]

print('ТЕСТОВАЯ ВЫБОРКА:\n')
print(f'Пользователей:      {users}')
print(f'Уникальных фильмов: {movies}')
print(f'Голосов:            {votes}')

In [ ]:
# считаем квадратичную ошибку оценки vote, по рейтингу и предсказанной оценке
result['squer_error_rating'] = (result['rating'] - result['vote'])**2
result['squer_error_predict'] = (result['vote_predict'] - result['vote'])**2
result

In [ ]:
# считаем rmse по рейтингу и предсказанию
# объединяем все ошибки, делим на их количество, берем корень
rmse_rating = np.sqrt( sum(result['squer_error_rating'])  / result.shape[0])
rmse_predict = np.sqrt( sum(result['squer_error_predict'])  / result.shape[0])

In [ ]:
print('TRAIN')
print(f'  users:          {train_users}')
print(f'  movies:         {train_movies}')
print(f'  votes:          {train_votes}')
print(f'  sparse_matrix:  {round(sparse_matrix,3)}')
print('TEST')
print(f'  users:          {users}')
print(f'  movies:         {movies}')
print(f'  votes:          {votes}')
print('INFO')
print(f'  K:              {K}')
print(f'  steps:          {steps}')
print(f'  lr:             {lr}')
print(f'  lambda:         {lambda_}')
print(f'  loss_model:     {round(loss,4)}')
print(f'  rmse_rating:    {round(rmse_rating,4)}')
print(f'  rmse_predict:   {round(rmse_predict,4)}')

# Краткий пересказ проекта

- Все данные парсились с КиноПоиска, и собраны в файле `data_kp_total.csv`  

- В ходе обработки данных были исключены все оценки равные единицы  
- Были исключены все фильмы у которых было менее 5 оценок

- Все необходимые данные были собраны в переменной `df`
- В итоге вышло 309 пользователя, и 4495 уникальных фильма

- Данные были разделены на `train` и `test` выборки, в соотношение 1/9
- В тестовой выборке 10877 оценок
- В тренеровочной 97892 оценок

- Тренировочные данные были трансформированы в сводную таблицу `pivot`, где:
 - строки – это пользователи
 - столбцы – это фильмы
 - значения – это оценки фильмам

- На основе сводной таблицы была создана матрица рейтингов `R`
- Уровень разряженности матрицы `R` составил - 0.07%

- Была создана бинарная матрица `R_binary`, оценивал ли пользователь фильм (True) или нет (False)
- Количество скрытых факторов `K` было установлено равным 80
- Матрица пользователь-фактор `P` и матрица фактор-элемент `Q` инициализированы случайными значениями

- Для построения предсказательной модели применяем **матричную факторизацию**
- Основная идея которой заключается в разложении исходной матрицы рейтингов `R` на две матрицы меньшей размерности `P` и `Q` так, чтобы их произведение приближало исходную матрицу `R`
- Для нахождения оптимальных значений матриц `P` и `Q` использовали метод градиентного спуска

- Обучение модели:
  - Количество шагов для обучения `steps` установлено – 400
  - Скорость обучения `lr` – 0.001
  - Параметр регуляризации `lambda_` был задан – 0.015


- В качестве оценки качества модели был выбран критерий MSE - средне квадратичная ошибка
- Входе обучения ошибка дошла до 0.54

- Получили матрицу предсказаний `approx`
- Заменили в `approx` оценки, которые были в исходной матрицы, используя `R_binary`
- Матрицу предсказаний `approx` конвертировали в класcический DataFrame - `model`
- `model` развернули в вертикальную таблицу `model_long`, с колонками:
  - `user_name`
  - `movie_ru`
  - `vote_predict`

- Объеденили тестовую выборку `test` с  `model_long`
- Получили таблицу `result`
- Посчитали для каждого пользователя `squer_error_rating` / `squer_error_predict`
- Расчитали `rmse_rating` / `rmse_predict`

# Итог

Проект заключался в том чтобы создать модель рекомендаций, которая сможет предсказывать оценки и рекомендовать фильмы пользователям, на основе их предыдущих оценок.


В ходе исследований получили интересные результаты:  
 - матричная факторизация действительно предсказывает оценку по фильмам
 - предсказывает ни хуже, чем рейтинг КиноПоиска, который по сути является средним значением генеральной совокупности всех голосовавших  
 - RMSE по рейтингу КиноПоиска составил 1.3337  
 - RMSE по предсказаной оценке составил 1.1791  
 - то есть нам хватило малого количества пользователей, и их оценок, чтобы воссоздать истинные рейтинги фильмов  

 - имея сильно разряженную матрицу в ~7%, используя матричную факторизацию, можно заполнить всю оставшуюся матрицу значениями, которые будут стремиться к истинным значениям
 - если переложить этот опыт на какой-нибудь бизнес, в котором нет заранее известного среднего значения *(рейтинга КП)*, то имея малый массив данных, выставив некоторые ограничения, можно получить достаточно точные предсказания по продукту
 - можно построить рекомендательную систему